In [ ]:
!pip install kaggle
from google.colab import files
files.upload()  # Upload kaggle.json here
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"
!kaggle competitions download -c aoml-m-2-test-feb-2025

Saving kaggle.json to kaggle.json
 75% 4.00M/5.35M [00:01<00:00, 4.50MB/s]
100% 5.35M/5.35M [00:01<00:00, 4.18MB/s]


In [ ]:
!unzip aoml-m-2-test-feb-2025.zip

Archive:  aoml-m-2-test-feb-2025.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
!pip install catboost
import numpy as np
import pandas as pd
import os
import zipfile
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, VotingRegressor, StackingRegressor, BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.2 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
from sklearn.impute import SimpleImputer
# Load dataset
data = pd.read_csv("train.csv")
print(data.head())

# Define target variable
target = "output_electricity_generation"
X = data.drop(columns=[target])
y = data[target]

# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=["object"]).columns
numerical_cols = X.select_dtypes(include=["number"]).columns

# Handle missing values
cat_imputer = SimpleImputer(strategy="most_frequent")
num_imputer = SimpleImputer(strategy="median")

X[categorical_cols] = cat_imputer.fit_transform(X[categorical_cols])
X[numerical_cols] = num_imputer.fit_transform(X[numerical_cols])

# Apply one-hot encoding
X = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

   uid       day  hour  minute    C_motion  feed_water_motion  faucet_hole  \
0    0  Saturday   7.0     NaN  387.374315        2681.205431     0.678352   
1    1  Saturday  12.0    28.0  238.153011        1604.213100     0.679706   
2    2    Friday  15.0    13.0  338.270228        2398.561685     0.662971   
3    3  Saturday  14.0     NaN  254.294492        1620.034059     0.679977   
4    4  Saturday  18.0    16.0  203.189410        1485.672389     0.686342   

   vapour_pressure  vapour_enthalpy  vapour_pressure_at_division  \
0        25.050893      2775.784134                    26.465138   
1        15.241147      2903.619864                          NaN   
2        23.078376      2796.443584                    24.316160   
3        15.054982      2818.023511                    15.895513   
4        12.963420      2816.144355                    13.682371   

   vapour_motion  feed_water_enth  vapour_temperature  \
0    2682.758692      1289.082351          601.850322   
1    160

In [ ]:
# Define models
models = {
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    "AdaBoost": AdaBoostRegressor(n_estimators=100, random_state=42),
    "CatBoost": CatBoostRegressor(iterations=500, depth=6, learning_rate=0.1, verbose=0),
    "XGBoost": XGBRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    "LightGBM": LGBMRegressor(n_estimators=100, random_state=42, n_jobs=-1)
}

In [ ]:
# Identify categorical columns
categorical_cols = X.select_dtypes(include=["object"]).columns

# Apply one-hot encoding
X = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [ ]:
# Train individual models
for name, model in models.items():
    print(f"Training {name}...")
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    rmse = mean_squared_error(y_test, preds) ** 0.5  # Manually compute RMSE
    print(f"{name} RMSE: {rmse:.4f}\n")


Training Decision Tree...
Decision Tree RMSE: 2.6778

Training Random Forest...
Random Forest RMSE: 2.9821

Training AdaBoost...
AdaBoost RMSE: 31.5827

Training CatBoost...
CatBoost RMSE: 4.2413

Training XGBoost...
XGBoost RMSE: 5.4901

Training LightGBM...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002528 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2636
[LightGBM] [Info] Number of data points in the train set: 40320, number of used features: 13
[LightGBM] [Info] Start training from score 832.300201
LightGBM RMSE: 4.2479



In [ ]:
# Bagging
bagging_model = BaggingRegressor(estimator=DecisionTreeRegressor(), n_estimators=50, random_state=42, n_jobs=-1)
bagging_model.fit(X_train, y_train)
bagging_preds = bagging_model.predict(X_test)
print(f"Bagging RMSE: {mean_squared_error(y_test, bagging_preds) ** 0.5:.4f}")

Bagging RMSE: 2.6833


In [ ]:
# Boosting
boosting_model = AdaBoostRegressor(n_estimators=100, random_state=42)
boosting_model.fit(X_train, y_train)
boosting_preds = boosting_model.predict(X_test)
print(f"Boosting RMSE: {mean_squared_error(y_test, boosting_preds) ** 0.5:.4f}")

Boosting RMSE: 31.5827


In [ ]:
# Voting
voting_model = VotingRegressor(estimators=[
    ("RF", models["Random Forest"]),
    ("XGB", models["XGBoost"]),
    ("CatBoost", models["CatBoost"])
])
voting_model.fit(X_train, y_train)
voting_preds = voting_model.predict(X_test)
print(f"Voting RMSE: {mean_squared_error(y_test, voting_preds) ** 0.5:.4f}")


Voting RMSE: 3.1458


In [ ]:
from sklearn.linear_model import Ridge
# Stacking (optimized)
stacking_model = StackingRegressor(
    estimators=[("RF", models["Random Forest"]), ("XGB", models["XGBoost"]), ("LGBM", models["LightGBM"])],
    final_estimator=Ridge(alpha=1.0),  # Faster and simpler final estimator
    n_jobs=-1
)
stacking_model.fit(X_train, y_train)
stacking_preds = stacking_model.predict(X_test)
print(f"Stacking RMSE: {mean_squared_error(y_test, stacking_preds) ** 0.5:.4f}")

Stacking RMSE: 3.6825


In [ ]:
# Blending
train_preds = (0.4 * models["Random Forest"].predict(X_test) +
               0.3 * models["XGBoost"].predict(X_test) +
               0.3 * models["CatBoost"].predict(X_test))
print(f"Blending RMSE: {mean_squared_error(y_test, train_preds) ** 0.5:.4f}")


Blending RMSE: 3.0138
